In [1]:
pip install fsspec

Note: you may need to restart the kernel to use updated packages.


In [70]:
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

import dask.bag as db
import json
import pandas as pd
import numpy as np

docs = db.read_text('arxiv-metadata-oai-snapshot.json').map(json.loads)

In [37]:
# The dataset is very huge. Not sure if the whole set can be used. I start prototyping with a subset of the data so it's easyer to handel:
# This procedure was recommended in the ArXiv dataset itself

get_latest_version = lambda x: x['versions'][-1]['created']


# get only necessary fields of the metadata file
trim = lambda x: {'title': x['title'],
                  'category':x['categories'].split(' '),}

# filter for papers published on or after 2021-01-01
columns = ['id','category','abstract']
docs_df = (docs.filter(lambda x: int(get_latest_version(x).split(' ')[3]) > 2020)
           .map(trim).
           compute())

In [38]:
docs_df = pd.DataFrame(docs_df)

In [39]:
docs_df.to_csv("trimmed_arxiv_docs.csv", index=False)

In [40]:
df = pd.read_csv("./trimmed_arxiv_docs.csv")

In [91]:
titles = df['title']
ml_identity = []
for i in range(0, len(df['category'])):
    if 'cs.LG' in df['category'][i]:
        ml_identity.append(1)
    else:
        ml_identity.append(0)
ml_identity = np.array(ml_identity)

In [124]:
n = 0
for i in range(0, len(ml_identity)):
    if ml_identity[i] == 1:
        n += 1
print('There have been', n, 'papers in Machine learning since 2021')

There have been 37088 papers in Machine learning since 2021
